<a href="https://colab.research.google.com/github/pandemic-tracking/global-vaccine/blob/main/WHO_Data_Lags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WHO Data Lags

In [17]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [18]:


prefix = "************\n"
postfix = "\n************"
# get WHO data
who_data = pd.read_csv("https://covid19.who.int/who-data/vaccination-data.csv")

# drop columns we aren't looking at right now
who_data = who_data.drop(columns=[
    'WHO_REGION','TOTAL_VACCINATIONS_PER100', 'PERSONS_VACCINATED_1PLUS_DOSE_PER100','PERSONS_FULLY_VACCINATED_PER100',
    'VACCINES_USED', 'FIRST_VACCINE_DATE', 'NUMBER_VACCINES_TYPES_USED', 'PERSONS_BOOSTER_ADD_DOSE_PER100',
    'PERSONS_BOOSTER_ADD_DOSE','PERSONS_VACCINATED_1PLUS_DOSE','PERSONS_FULLY_VACCINATED'
    ])

# add date diff column
who_data['date_diff'] = (pd.to_datetime("today") - pd.to_datetime(who_data['DATE_UPDATED'])).dt.days

print(prefix,'Countries with reporting more than 60 days lagged: ' , len( who_data.loc[who_data['date_diff'] > 60]),postfix)
who_data.loc[who_data['date_diff'] > 60]

************
 Countries with reporting more than 60 days lagged:  34 
************


,COUNTRY,ISO3,DATA_SOURCE,DATE_UPDATED,TOTAL_VACCINATIONS,date_diff
9,Armenia,ARM,REPORTING,2022-05-22,2150112.0,131
26,"Bonaire, Sint Eustatius and Saba",BES,OWID,2021-09-01,35845.0,394
27,Bosnia and Herzegovina,BIH,REPORTING,2022-01-29,1924950.0,244
45,Comoros,COM,REPORTING,2022-07-31,804403.0,61
46,Congo,COG,REPORTING,2022-07-31,833210.0,61
47,Cook Islands,COK,REPORTING,2022-05-10,39612.0,143
63,Equatorial Guinea,GNQ,REPORTING,2022-07-24,488738.0,68
64,Eritrea,ERI,REPORTING,2022-07-03,NaN,89
67,Ethiopia,ETH,REPORTING,2022-07-31,52509414.0,61
68,Falkland Islands (Malvinas),FLK,REPORTING,2022-06-21,9390.0,101


# Get Population Sizes

In [21]:
# get population size from world bank
from zipfile import ZipFile
import io
from urllib.request import urlopen
import pandas as pd

r = urlopen("https://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=csv").read()
file = ZipFile(io.BytesIO(r))
file_name = file.filelist[1].filename # this is kind of fragile, I am not sure if it will always be in this position..
pop_df = pd.read_csv(file.open(file_name), skiprows=4)[["Country Name", "Country Code","2021"]]


# Merge Population And Vaccine Administration

In [20]:
vax_with_pop = pd.merge(who_data,pop_df,left_on='ISO3',right_on='Country Code')

print('Countries with more than 1 million pop and reporting more than 60 days old')
# Force Eritrea to appear even though world bank is missing its population (3.63MM)
vax_with_pop.loc[((vax_with_pop['date_diff'] > 60) & (vax_with_pop['2021'] > 1000000)) | (vax_with_pop['ISO3'] == 'ERI')]

Countries with more than 1 million pop and reporting more than 60 days old


,COUNTRY,ISO3,DATA_SOURCE,DATE_UPDATED,TOTAL_VACCINATIONS,date_diff,Country Name,Country Code,2021
8,Armenia,ARM,REPORTING,2022-05-22,2150112.0,131,Armenia,ARM,2968128.0
24,Bosnia and Herzegovina,BIH,REPORTING,2022-01-29,1924950.0,244,Bosnia and Herzegovina,BIH,3263459.0
43,Congo,COG,REPORTING,2022-07-31,833210.0,61,"Congo, Rep.",COG,5657017.0
59,Equatorial Guinea,GNQ,REPORTING,2022-07-24,488738.0,68,Equatorial Guinea,GNQ,1449891.0
60,Eritrea,ERI,REPORTING,2022-07-03,NaN,89,Eritrea,ERI,NaN
63,Ethiopia,ETH,REPORTING,2022-07-31,52509414.0,61,Ethiopia,ETH,117876226.0
93,Israel,ISR,REPORTING,2022-06-05,15026951.0,117,Israel,ISR,9364000.0
129,Myanmar,MMR,REPORTING,2022-06-18,64549994.0,104,Myanmar,MMR,54806014.0
168,Serbia,SRB,REPORTING,2022-06-12,6708547.0,110,Serbia,SRB,6844078.0
181,Sudan,SDN,REPORTING,2022-05-30,8179010.0,123,Sudan,SDN,44909351.0
